In [1]:
import cv2
import sys
import os
import time
import dlib
import random
import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA
from sklearn.cross_validation  import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from sklearn.cross_validation  import train_test_split


C:\Users\test\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\test\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
#使用dlib库进行人脸检测
cascPath = "haarcascade_frontalface_default.xml"
dataPath='D:\\code\\pycode\\faceRecognition\\database\\'
patchPath='D:\\code\\pycode\\faceRecognition\\faceData\\'
faceCascade = cv2.CascadeClassifier(cascPath)
for personFolder in os.listdir(dataPath):
    personPath=os.path.join(dataPath,personFolder)
    for fileName in os.listdir(personPath):
        #遍历文件夹下所有图片
        image = cv2.imread(os.path.join(personPath,fileName))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #将图片转为灰度模式
        faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30, 30))
        if(len(faces)==1):
            x,y,w,h=list(faces[0])
            facePatch=image[y:y+h,x:x+w]
            if(not os.path.exists(os.path.join(patchPath,personFolder))):
                os.mkdir(os.path.join(patchPath,personFolder)) 
            #保存所有检测到的人脸
            cv2.imwrite(os.path.join(patchPath+personFolder,fileName),facePatch)

In [ ]:
#使用ImageDataGenerator进行数据增强
detector = dlib.get_frontal_face_detector()
dataPath='D:\\code\\pycode\\faceRecognition\\database\\'
patchPath='D:\\code\\pycode\\faceRecognition\\faceData1\\'
detector = dlib.get_frontal_face_detector()
datagen = ImageDataGenerator(
        rotation_range=20,
#        width_shift_range=0.2,
#        height_shift_range=0.2,
        rescale=None,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        data_format='channels_last',
        cval=0,
        #channel_shift_range=0,
        vertical_flip=False)
for personFolder in os.listdir(dataPath):
    personPath=os.path.join(dataPath,personFolder)
    for fileName in os.listdir(personPath):
        image = cv2.imread(os.path.join(personPath,fileName))
        dets = detector(image, 1)
        for i, d in enumerate(dets):
            print("person{} face{} Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(personFolder,fileName[-5:-4],i, d.left(), d.top(), d.right(), d.bottom()))
            facePatch=image[d.top():d.bottom(),d.left():d.right()]
            cv2.imwrite(os.path.join(patchPath+personFolder,fileName),facePatch)
            
            facePatch=img_to_array(facePatch)
            facePatch = facePatch.reshape((1,) + facePatch.shape)
            if(not os.path.exists(os.path.join(patchPath,personFolder))):
                os.mkdir(os.path.join(patchPath,personFolder)) 
                
            i = 0
            for batch in datagen.flow(facePatch, batch_size=5,
                                      save_to_dir=None#os.path.join(patchPath,personFolder), 
                                      #save_prefix=personFolder, 
                                      #save_format='jpeg'
                                      ):
                cv2.imwrite(os.path.join(patchPath+personFolder,fileName[:-4]+str(i)+'.JPG'),batch[0].astype(int))
                i += 1
                if i > 5:
                    break  # 否则生成器会退出循环
            


In [4]:
#读取所有人脸数据，并使用PCA进行降维
trainData=[[]]
label=[]
dataPath='D:\\code\\pycode\\faceRecognition\\faceData1\\'
for dataLabel in os.listdir(dataPath):
    folderPath=os.path.join(dataPath,dataLabel)
    for fileName in os.listdir(folderPath):
        image=cv2.imread(os.path.join(folderPath,fileName))
        gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        imageArray=np.array(cv2.resize(gray_image,(101,101))).reshape([101*101])/255
        label.append(int(dataLabel))
        if(trainData==[[]]):
            trainData = [imageArray]
            continue
        trainData = np.concatenate((trainData, [imageArray]),axis=0)
pca = PCA(n_components=0.95)
pca.fit(trainData)
#print ("explained_variance_ratio_",pca.explained_variance_ratio_)
#print ("explained_variance_",pca.explained_variance_)
print ("pca.n_components_",pca.n_components_)
label=np.array(label)
trainData=pca.transform(trainData)
    train_images, test_images, train_labels, test_labels = train_test_split(trainData, label, test_size = 0.1, random_state = random.randint(0, 100)) 

C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  # This is added back by InteractiveShellApp.init_path()


explained_variance_ [9.30563394e+01 2.59330455e+01 1.41939514e+01 8.40064569e+00
 7.82357035e+00 6.42211011e+00 6.13298868e+00 4.03662275e+00
 3.64808265e+00 2.95069853e+00 2.72460899e+00 2.18832621e+00
 2.07116385e+00 1.88104333e+00 1.71905881e+00 1.66628237e+00
 1.39929170e+00 1.25982322e+00 1.14435455e+00 1.07452002e+00
 1.02019881e+00 9.84561914e-01 9.21918097e-01 8.95024320e-01
 8.41218152e-01 7.96008563e-01 7.83868748e-01 7.36044410e-01
 7.10644629e-01 6.59112705e-01 6.50906526e-01 6.01274336e-01
 5.80219340e-01 5.63628472e-01 5.34013969e-01 5.15436771e-01
 4.84377142e-01 4.70503563e-01 4.55146570e-01 4.42822087e-01
 4.14670610e-01 4.00608457e-01 3.95222584e-01 3.87632994e-01
 3.74686138e-01 3.64963239e-01 3.54545247e-01 3.46966623e-01
 3.34975085e-01 3.32642584e-01 3.09766357e-01 3.07493404e-01
 3.03037957e-01 2.94842128e-01 2.89603262e-01 2.87722256e-01
 2.78269778e-01 2.70458588e-01 2.65288739e-01 2.61039397e-01
 2.49694320e-01 2.40479758e-01 2.37044316e-01 2.33929066e-01
 2.2

In [5]:
#使用SVM进行分类
svc = SVC(gamma='auto')
svc.fit(train_images,train_labels)   #训练模型
test_pred=svc.predict(test_images)   #用测试数据来做预测
print(classification_report(test_labels, test_pred))

             precision    recall  f1-score   support

          1       0.50      1.00      0.67         1
          2       1.00      0.30      0.46        10
          3       1.00      0.17      0.29         6
          4       1.00      0.25      0.40         4
          5       0.00      0.00      0.00         2
          6       0.33      0.20      0.25         5
          7       0.14      0.50      0.22         2
          8       0.33      0.20      0.25         5
          9       0.33      0.50      0.40         2
         10       0.00      0.00      0.00         3
         11       0.50      0.17      0.25         6
         12       0.33      1.00      0.50         2
         13       0.67      0.40      0.50         5
         14       0.00      0.00      0.00         4
         15       0.00      0.00      0.00         1
         16       0.00      0.00      0.00         1
         17       0.00      0.00      0.00         1
         18       1.00      0.50      0.67   

C:\Users\test\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\test\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
clf = RandomForestClassifier(n_jobs=111)
clf.fit(train_images,train_labels) 
test_pred=clf.predict(test_images)  #用测试数据来做预测
print(classification_report(test_labels,test_pred))

In [ ]:
IMAGE_SIZE = 101
images = []
labels = []
MODEL_PATH='D:\\code\\pycode\\faceRecognition\\models.h'
def read_path(path_name):    
    for dir in os.listdir(path_name):
        picFolderPath=os.path.join(path_name,dir)
        print(dir+' '+str(len(os.listdir(picFolderPath))))
        for picname in os.listdir(picFolderPath):
            if(picname.endswith('JPG')):
                labels.append(int(dir))
                image = cv2.imread(os.path.join(picFolderPath,picname))
                image=cv2.resize(image,(101, 101), interpolation=cv2.INTER_CUBIC)
                images.append(image)
    return images,labels

In [ ]:
#从指定路径读取训练数据
def load_dataset(path_name):
    images,labels = read_path(path_name)    
    
    #将输入的所有图片转成四维数组，尺寸为(图片数量*IMAGE_SIZE*IMAGE_SIZE*3)
    #我和闺女两个人共1200张图片，IMAGE_SIZE为64，故对我来说尺寸为1200 * 64 * 64 * 3
    #图片为64 * 64像素,一个像素3个颜色值(RGB)
    images = np.array(images)
    print(images.shape)    
    return images, labels

In [ ]:
class Dataset:
    def __init__(self, path_name):
        #训练集
        self.train_images = None
        self.train_labels = None
        
        #验证集
        self.valid_images = None
        self.valid_labels = None
        
        #测试集
        self.test_images  = None            
        self.test_labels  = None
        
        #数据集加载路径
        self.path_name    = path_name
        
        #当前库采用的维度顺序
        self.input_shape = None
        
    #加载数据集并按照交叉验证的原则划分数据集并进行相关预处理工作
    def load(self, img_rows = 101, img_cols = 101, 
             img_channels = 3, nb_classes = 111):
        #加载数据集到内存
        images, labels = load_dataset(self.path_name)  
        train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size = 0.2, random_state = random.randint(0, 100)) 
        valid_images, test_images, valid_labels, test_labels = train_test_split(test_images, test_labels, test_size = 0.5, random_state = random.randint(0, 100)) 
        
        #train_images, valid_images, train_labels, valid_labels = train_test_split(images, labels, test_size = 0.3, random_state = random.randint(0, 100))        
        #_, test_images, _, test_labels = train_test_split(images, labels, test_size = 0.5, random_state = random.randint(0, 100))                
        
        #当前的维度顺序如果为'th'，则输入图片数据时的顺序为：channels,rows,cols，否则:rows,cols,channels
        #这部分代码就是根据keras库要求的维度顺序重组训练数据集
        if K.image_dim_ordering() == 'th':
            train_images = train_images.reshape(train_images.shape[0], img_channels, img_rows, img_cols)
            valid_images = valid_images.reshape(valid_images.shape[0], img_channels, img_rows, img_cols)
            test_images = test_images.reshape(test_images.shape[0], img_channels, img_rows, img_cols)
            self.input_shape = (img_channels, img_rows, img_cols)            
        else:
            train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, img_channels)
            valid_images = valid_images.reshape(valid_images.shape[0], img_rows, img_cols, img_channels)
            test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, img_channels)
            self.input_shape = (img_rows, img_cols, img_channels)            
            
        #输出训练集、验证集、测试集的数量
        print(train_images.shape[0], 'train samples')
        print(valid_images.shape[0], 'valid samples')
        print(test_images.shape[0], 'test samples')
    
        #我们的模型使用categorical_crossentropy作为损失函数，因此需要根据类别数量nb_classes将
        #类别标签进行one-hot编码使其向量化，在这里我们的类别只有两种，经过转化后标签数据变为二维
#            train_labels = np_utils.to_categorical(train_labels, nb_classes)                        
#            valid_labels = np_utils.to_categorical(valid_labels, nb_classes)            
#            test_labels = np_utils.to_categorical(test_labels, nb_classes)                        
    
        #像素数据浮点化以便归一化
        train_images = train_images.astype('float32')            
        valid_images = valid_images.astype('float32')
        test_images = test_images.astype('float32')
        
        #将其归一化,图像的各像素值归一化到0~1区间
        train_images /= 255
        valid_images /= 255
        test_images /= 255            
        lb=LabelBinarizer().fit(np.array(range(0,nb_classes)))
        
        self.train_images = train_images
        self.valid_images = valid_images
        self.test_images  = test_images
        self.train_labels = lb.transform(train_labels)
        self.valid_labels = lb.transform(valid_labels)
        self.test_labels  = lb.transform(test_labels)

In [ ]:
class Model:
    def __init__(self):
        self.model = None 
        self.num_classes = 111
        self.weight_decay = 0.0005
        self.x_shape = [224,224,3]
        
    #建立模型
    def build_model(self, dataset, nb_classes = 111):
        #构建一个空的网络模型，它是一个线性堆叠模型，各神经网络层会被顺序添加，专业名称为序贯模型或线性堆叠模型
        self.model = Sequential() 
        weight_decay=self.weight_decay
        #以下代码将顺序添加CNN网络需要的各层，一个add就是一个网络层
        self.model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.3))

        self.model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())

        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.4))

        self.model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())

        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.4))

        self.model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.4))

        self.model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())

        self.model.add(MaxPooling2D(pool_size=(2, 2)))


        self.model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.4))

        self.model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.4))

        self.model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())

        self.model.add(MaxPooling2D(pool_size=(2, 2)))


        self.model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.4))

        self.model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.4))

        self.model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())

        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.5))

        self.model.add(Flatten())
        self.model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())

        self.model.add(Dropout(0.5))
        self.model.add(Dense(nb_classes))
        self.model.add(Activation('softmax'))

        #输出模型概况
        self.model.summary()
        #训练模型
    def train(self, dataset, batch_size = 128, nb_epoch = 500, data_augmentation = False):        
        sgd = SGD(lr = 0.01, decay = 1e-6, 
                  momentum = 0.9, nesterov = True) #采用SGD+momentum的优化器进行训练，首先生成一个优化器对象  
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=sgd,
                           metrics=['accuracy'])   #完成实际的模型配置工作
        
        #不使用数据提升，所谓的提升就是从我们提供的训练数据中利用旋转、翻转、加噪声等方法创造新的
        #训练数据，有意识的提升训练数据规模，增加模型训练量+
        if not data_augmentation:            
            self.model.fit(dataset.train_images,
                           dataset.train_labels,
                           batch_size = batch_size,
                           nb_epoch = nb_epoch,
                           validation_data = (dataset.valid_images, dataset.valid_labels),
                           shuffle = True)
        #使用实时数据提升
        else:            
            #定义数据生成器用于数据提升，其返回一个生成器对象datagen，datagen每被调用一
            #次其生成一组数据（顺序生成），节省内存，其实就是python的数据生成器
            datagen = ImageDataGenerator(
                featurewise_center = False,             #是否使输入数据去中心化（均值为0），
                samplewise_center  = False,             #是否使输入数据的每个样本均值为0
                featurewise_std_normalization = False,  #是否数据标准化（输入数据除以数据集的标准差）
                samplewise_std_normalization  = False,  #是否将每个样本数据除以自身的标准差
                zca_whitening = False,                  #是否对输入数据施以ZCA白化
                rotation_range = 20,                    #数据提升时图片随机转动的角度(范围为0～180)
                width_shift_range  = 0.2,               #数据提升时图片水平偏移的幅度（单位为图片宽度的占比，0~1之间的浮点数）
                height_shift_range = 0.2,               #同上，只不过这里是垂直
                horizontal_flip = True,                 #是否进行随机水平翻转
                vertical_flip = False)                  #是否进行随机垂直翻转

            #计算整个训练样本集的数量以用于特征值归一化、ZCA白化等处理
            datagen.fit(dataset.train_images)                        

            #利用生成器开始训练模型
            self.model.fit_generator(datagen.flow(dataset.train_images, dataset.train_labels,
                                                   batch_size = batch_size),
                                     samples_per_epoch = dataset.train_images.shape[0],
                                     nb_epoch = nb_epoch,
                                     validation_data = (dataset.valid_images, dataset.valid_labels))
    def evaluate(self, dataset):
        score = self.model.evaluate(dataset.test_images, dataset.test_labels, verbose = 1)
        print("%s: %.2f%%" % (self.model.metrics_names[1], score[1] * 100))
    def save_model(self, file_path = MODEL_PATH):
         self.model.save(file_path)
    def load_model(self, file_path = MODEL_PATH):
        self.model = load_model(file_path)

In [ ]:
dataset = Dataset('D:\\code\pycode\\faceRecognition\\faceData1\\')    
dataset.load()
model = Model()
model.build_model(dataset)
model.train(dataset)    
model.save_model()
model.evaluate(dataset)